In [ ]:
# Quantum Network Simulation for Entanglement Distribution
import sys
import netsquid as ns
import pandas
from netsquid.components.qprocessor import QuantumProcessor, PhysicalInstruction
from netsquid.nodes import Node, Connection, Network
from netsquid.protocols.protocol import Signals
from netsquid.protocols.nodeprotocols import NodeProtocol
from netsquid.components.qchannel import QuantumChannel
from netsquid.components.cchannel import ClassicalChannel
from netsquid.components.qsource import QSource, SourceStatus
from netsquid.qubits.state_sampler import StateSampler
from netsquid.components.qprogram import QuantumProgram
from netsquid.components.models.qerrormodels import FibreLossModel, DepolarNoiseModel, DephaseNoiseModel
from netsquid.components.models.delaymodels import FibreDelayModel, FixedDelayModel
from netsquid.util.datacollector import DataCollector
import pydynaa
from netsquid.qubits import ketstates as ks
from netsquid.qubits import qubitapi as qapi
from netsquid.components import instructions as instr
from netsquid.components.instructions import Instruction
from netsquid.qubits.operators import Operator, I, X, Y
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

# Module imports verification
print(sys.path)

__all__ = [
    "ExternalEntanglingConnection",
    "ClassicalConnection",
    "InitStateProgram",
    "BellMeasurementProgram",
    "BellMeasurementProtocol",
    "CorrectionProtocol",
    "create_processor",
    "create_client_processor",
    "example_network_setup",
    "example_sim_setup",
    "run_dis_experiment",
    "create_dis_plot",
]

# Protocol completion flags
global_finished_bell = False
global_finished_correction = False

def memory_is_available(mem_position):
    """Check if the memory position is empty and available for use"""
    return mem_position.is_empty

class ExternalEntanglingConnection(Connection):
    """
    Connection with quantum channels only, adding QSource as a subcomponent.
    This allows for external control of entanglement generation.
    """
    def __init__(self, length, name="ExternalEntanglingConnection", fidelity=1.0):
        super().__init__(name=name)

        # Configure quantum channels
        qchannel_c2a = QuantumChannel(
            "qchannel_C2A",
            length=length / 2,
            models={
                "quantum_loss_model": FibreLossModel(p_loss_init=0, p_loss_length=0),
                "quantum_noise_model": DepolarNoiseModel(depolar_rate=0)
            }
        )
        qchannel_c2b = QuantumChannel(
            "qchannel_C2B",
            length=length / 2,
            models={
                "quantum_loss_model": FibreLossModel(p_loss_init=0, p_loss_length=0),
                "quantum_noise_model": DepolarNoiseModel(depolar_rate=0)
            }
        )
        # Add channels as subcomponents
        self.add_subcomponent(qchannel_c2a, forward_output=[("A", "recv")])
        self.add_subcomponent(qchannel_c2b, forward_output=[("B", "recv")])

        # Create QSource with specified fidelity
        qsource = create_external_qsource(name="AliceQSource", fidelity=fidelity)
        self.add_subcomponent(qsource)

        # Connect QSource ports to quantum channels
        qsource.ports["qout0"].connect(qchannel_c2a.ports["send"])
        qsource.ports["qout1"].connect(qchannel_c2b.ports["send"])

class ExternalSourceProtocol(NodeProtocol):
    """
    Protocol to control external entanglement source.
    Waits for both Alice and Bob's memories to be available before triggering the source.
    """

    def __init__(self, node, qsource, other_node, mem_pos_a=3, mem_pos_b=0,
                 base_delay=1e9/200, extra_delay=1e6, max_retries=3000000):
        """
        Parameters
        ----------
        node : Node
            Protocol node (Alice)
        qsource : QSource
            EXTERNAL mode QSource
        other_node : Node
            Node to check memory status (Bob)
        mem_pos_a : int
            Alice's memory position index
        mem_pos_b : int
            Bob's memory position index
        base_delay : float
            Initial wait time [ns]
        extra_delay : float
            Additional wait time if memories are occupied [ns]
        max_retries : int
            Maximum number of retry attempts
        """
        super().__init__(node)
        self.qsource = qsource
        self.other_node = other_node
        self.mem_pos_a = mem_pos_a
        self.mem_pos_b = mem_pos_b
        self.base_delay = base_delay
        self.extra_delay = extra_delay
        self.max_retries = max_retries

    def run(self):
        # Main protocol loop
        while True:
            yield self.await_timer(self.base_delay)

            retries = 0
            while True:
                # Check Alice's memory position
                mem_position_a = self.node.qmemory.mem_positions[self.mem_pos_a]
                # Check Bob's memory position
                mem_position_b = self.other_node.qmemory.mem_positions[self.mem_pos_b]

                if memory_is_available(mem_position_a) and memory_is_available(mem_position_b):
                    # Both memories available, trigger QSource
                    self.qsource.trigger()
                    break
                else:
                    if retries >= self.max_retries:
                        break
                    # Memories occupied, wait extra delay
                    retries += 1
                    yield self.await_timer(self.extra_delay)

class ClassicalConnection(Connection):
    """A connection that transmits classical messages in one direction, from A to B."""
    def __init__(self, length, name="ClassicalConnection"):
        super().__init__(name=name)
        self.add_subcomponent(
            ClassicalChannel("Channel_A2B", length=length,
                             models={"delay_model": FibreDelayModel(c=200000)}),
            forward_input=[("A", "send")],
            forward_output=[("B", "recv")]
        )

from netsquid.components.models.qerrormodels import DepolarNoiseModel
from netsquid.components.qprocessor import PhysicalInstruction
from netsquid.components import instructions as instr
from netsquid.components.models.qerrormodels import T1T2NoiseModel

def create_processor(dephase_rate=0.0039, T1=1e10, T2_ratio=0.1, sge=None, dge=None, gate_speed_factor=1.0, gate_durations=None):
    """
    Factory function to create quantum processors with configurable gate speeds
    
    Parameters
    ----------
    dephase_rate : float
        Dephasing rate for measurement operations
    T1 : float
        Relaxation time [ns]
    T2_ratio : float
        Ratio of T2/T1 for coherence time
    sge : float
        Single-gate error rate
    dge : float
        Double-gate error rate
    gate_speed_factor : float
        Factor to scale gate durations (>1 means faster)
    gate_durations : dict
        Custom gate durations in ns
    """
    if gate_durations is None:
        # Default gate execution times [ns]
        gate_durations = {
            instr.INSTR_INIT: 1000,
            instr.INSTR_H: 135000,
            instr.INSTR_X: 135000,
            instr.INSTR_Z: 135000,
            instr.INSTR_Y: 135000,
            instr.INSTR_S: 135000,
            instr.INSTR_ROT_X: 135000,
            instr.INSTR_ROT_Y: 135000,
            instr.INSTR_ROT_Z: 135000,
            instr.INSTR_CNOT: 600000,
            instr.INSTR_MEASURE: 200000
        }
    
    # Apply gate speed factor
    scaled_gate_durations = {k: v / gate_speed_factor for k, v in gate_durations.items()}
    
    # Memory noise model
    memory_noise_model = T1T2NoiseModel(T1=T1, T2=T1*T2_ratio)
    
    # Gate noise models
    gate_noise_models = {
        instr.INSTR_H: DepolarNoiseModel(depolar_rate=sge, time_independent=True),
        instr.INSTR_X: DepolarNoiseModel(depolar_rate=sge, time_independent=True),
        instr.INSTR_Z: DepolarNoiseModel(depolar_rate=sge, time_independent=True),
        instr.INSTR_Y: DepolarNoiseModel(depolar_rate=sge, time_independent=True),
        instr.INSTR_ROT_X: DepolarNoiseModel(depolar_rate=sge, time_independent=True),
        instr.INSTR_ROT_Z: DepolarNoiseModel(depolar_rate=sge, time_independent=True),
        instr.INSTR_ROT_Y: DepolarNoiseModel(depolar_rate=sge, time_independent=True),
        instr.INSTR_CNOT: DepolarNoiseModel(depolar_rate=dge, time_independent=True),
    }
    
    # Physical instructions setup
    physical_instructions = [
        PhysicalInstruction(instr.INSTR_INIT, duration=scaled_gate_durations[instr.INSTR_INIT], parallel=True),
        PhysicalInstruction(instr.INSTR_H, duration=scaled_gate_durations[instr.INSTR_H], parallel=True, topology=[0, 1, 2, 3],
                           quantum_noise_model=gate_noise_models.get(instr.INSTR_H, None)),
        PhysicalInstruction(instr.INSTR_X, duration=scaled_gate_durations[instr.INSTR_X], parallel=True, topology=[0, 1, 2, 3],
                           quantum_noise_model=gate_noise_models.get(instr.INSTR_X, None)),
        PhysicalInstruction(instr.INSTR_Z, duration=scaled_gate_durations[instr.INSTR_Z], parallel=True, topology=[0, 1, 2, 3]),
        PhysicalInstruction(instr.INSTR_Y, duration=scaled_gate_durations[instr.INSTR_Y], parallel=True, topology=[0, 1, 2, 3]),
        PhysicalInstruction(instr.INSTR_S, duration=scaled_gate_durations[instr.INSTR_S], parallel=True, topology=[0, 1, 2, 3]),
        PhysicalInstruction(instr.INSTR_ROT_X, duration=scaled_gate_durations[instr.INSTR_ROT_X], parallel=True, topology=[0, 1, 2, 3]),
        PhysicalInstruction(instr.INSTR_ROT_Y, duration=scaled_gate_durations[instr.INSTR_ROT_Y], parallel=True, topology=[0, 1, 2, 3]),
        PhysicalInstruction(instr.INSTR_ROT_Z, duration=scaled_gate_durations[instr.INSTR_ROT_Z], parallel=True, topology=[0, 1, 2, 3]),
        PhysicalInstruction(instr.INSTR_CNOT, duration=scaled_gate_durations[instr.INSTR_CNOT], parallel=True, topology=[(0, 1)]),
        PhysicalInstruction(instr.INSTR_CNOT, duration=scaled_gate_durations[instr.INSTR_CNOT], parallel=True, topology=[(0, 2)]),
        PhysicalInstruction(instr.INSTR_CNOT, duration=scaled_gate_durations[instr.INSTR_CNOT], parallel=True, topology=[(1, 2)]),
        PhysicalInstruction(instr.INSTR_CNOT, duration=scaled_gate_durations[instr.INSTR_CNOT], parallel=True, topology=[(2, 3)]),
        PhysicalInstruction(instr.INSTR_MEASURE, duration=scaled_gate_durations[instr.INSTR_MEASURE], parallel=False, topology=[0],
                           quantum_noise_model=DepolarNoiseModel(depolar_rate=dephase_rate, time_independent=True),
                           apply_q_noise_after=False),
        PhysicalInstruction(instr.INSTR_MEASURE, duration=scaled_gate_durations[instr.INSTR_MEASURE], parallel=False, topology=[1],
                           quantum_noise_model=DepolarNoiseModel(depolar_rate=dephase_rate, time_independent=True),
                           apply_q_noise_after=False),
        PhysicalInstruction(instr.INSTR_MEASURE, duration=scaled_gate_durations[instr.INSTR_MEASURE], parallel=False, topology=[2],
                           quantum_noise_model=DepolarNoiseModel(depolar_rate=dephase_rate, time_independent=True),
                           apply_q_noise_after=False),
        PhysicalInstruction(instr.INSTR_MEASURE, duration=scaled_gate_durations[instr.INSTR_MEASURE], parallel=False, topology=[3],
                           quantum_noise_model=DepolarNoiseModel(depolar_rate=dephase_rate, time_independent=True),
                           apply_q_noise_after=False)
    ]
    
    # Create quantum processor
    processor = QuantumProcessor("quantum_processor", num_positions=4,
                                 memory_noise_models=[memory_noise_model] * 4,
                                 phys_instructions=physical_instructions)
    return processor

def create_client_processor(dephase_rate=0.0039, T1=1e10, T2_ratio=0.1, sge=None, dge=None, gate_speed_factor=1.0, gate_durations=None):
    """
    Factory function to create client quantum processors with limited instruction set
    
    Parameters same as create_processor
    """
    if gate_durations is None:
        # Default gate execution times [ns]
        gate_durations = {
            instr.INSTR_INIT: 1000,
            instr.INSTR_H: 135000,
            instr.INSTR_X: 135000,
            instr.INSTR_Z: 135000,
            instr.INSTR_Y: 135000,
            instr.INSTR_S: 135000,
            instr.INSTR_ROT_X: 135000,
            instr.INSTR_ROT_Y: 135000,
            instr.INSTR_ROT_Z: 135000,
            instr.INSTR_CNOT: 600000,
            instr.INSTR_MEASURE: 200000
        }
    
    # Apply gate speed factor
    scaled_gate_durations = {k: v / gate_speed_factor for k, v in gate_durations.items()}
    
    # Memory noise model
    memory_noise_model = T1T2NoiseModel(T1=T1, T2=T1*T2_ratio)
    
    # Gate noise models
    gate_noise_models = {
        instr.INSTR_H: DepolarNoiseModel(depolar_rate=sge, time_independent=True),
        instr.INSTR_X: DepolarNoiseModel(depolar_rate=sge, time_independent=True),
        instr.INSTR_Z: DepolarNoiseModel(depolar_rate=sge, time_independent=True),
        instr.INSTR_Y: DepolarNoiseModel(depolar_rate=sge, time_independent=True),
        instr.INSTR_ROT_X: DepolarNoiseModel(depolar_rate=sge, time_independent=True),
        instr.INSTR_ROT_Z: DepolarNoiseModel(depolar_rate=sge, time_independent=True),
        instr.INSTR_ROT_Y: DepolarNoiseModel(depolar_rate=sge, time_independent=True)
    }
    
    # Client-specific physical instructions - only necessary operations
    physical_instructions = [
        # H gate required for ClientProgram
        PhysicalInstruction(instr.INSTR_H, duration=scaled_gate_durations[instr.INSTR_H], 
                          parallel=True, topology=[0],
                          quantum_noise_model=gate_noise_models.get(instr.INSTR_H)),
        
        # ROT_Z gate essential for rotations
        PhysicalInstruction(instr.INSTR_ROT_Z, duration=scaled_gate_durations[instr.INSTR_ROT_Z], 
                          parallel=True, topology=[0],
                          quantum_noise_model=gate_noise_models.get(instr.INSTR_ROT_Z)),
        
        # Other single-qubit gates for completeness
        PhysicalInstruction(instr.INSTR_X, duration=scaled_gate_durations[instr.INSTR_X], 
                          parallel=True, topology=[0],
                          quantum_noise_model=gate_noise_models.get(instr.INSTR_X)),
        
        PhysicalInstruction(instr.INSTR_Z, duration=scaled_gate_durations[instr.INSTR_Z], 
                          parallel=True, topology=[0],
                          quantum_noise_model=gate_noise_models.get(instr.INSTR_Z)),
        
        PhysicalInstruction(instr.INSTR_Y, duration=scaled_gate_durations[instr.INSTR_Y], 
                          parallel=True, topology=[0],
                          quantum_noise_model=gate_noise_models.get(instr.INSTR_Y)),
        
        # Measurement operation
        PhysicalInstruction(instr.INSTR_MEASURE, duration=scaled_gate_durations[instr.INSTR_MEASURE], 
                          parallel=False, topology=[0],
                          quantum_noise_model=DepolarNoiseModel(depolar_rate=dephase_rate, time_independent=True),
                          apply_q_noise_after=False)
    ]
    
    # Create client processor with single qubit
    processor = QuantumProcessor("client_quantum_processor", num_positions=1,
                                memory_noise_models=[memory_noise_model],
                                phys_instructions=physical_instructions)
    return processor

def create_external_qsource(name="ExtQSource", fidelity=1.0):
    """
    Factory function to create EXTERNAL mode QSource.
    For fidelity < 1.0, applies depolarizing noise to reduce entanglement quality.
    
    Parameters
    ----------
    name : str
        Name of the QSource
    fidelity : float
        Target fidelity of generated Bell pairs
    
    Returns
    -------
    QSource
        Configured quantum source in EXTERNAL trigger mode
    """
    from netsquid.qubits import ketstates as ks
    
    # Convert fidelity F to depolarizing probability p
    p_depol = 4/3 * (1 - fidelity)
    if p_depol < 0:
        p_depol = 0.0  # Protection against invalid fidelity values
    
    qsource = QSource(
        name=name,
        state_sampler=StateSampler([ks.b00], [1.0]),
        num_ports=2,
        status=SourceStatus.EXTERNAL,
        models={
            # Apply depolarizing noise at emission time
            "emission_noise_model": DepolarNoiseModel(
                depolar_rate=p_depol,
                time_independent=True  # Time-independent noise model
            )
        }
    )
    return qsource

# Quantum Program Definitions
class InitStateProgram(QuantumProgram):
    default_num_qubits = 3
    def program(self):
        q1, q2, q3 = self.get_qubit_indices(3)
        self.apply(instr.INSTR_INIT, q1)
        self.apply(instr.INSTR_INIT, q2)
        self.apply(instr.INSTR_INIT, q3)
        self.apply(instr.INSTR_ROT_Y, q1, angle=np.pi / 2)
        self.apply(instr.INSTR_X, q2)
        self.apply(instr.INSTR_ROT_X, q2, angle=-np.pi / 2)
        self.apply(instr.INSTR_CNOT, [q1, q2])
        self.apply(instr.INSTR_CNOT, [q2, q3])
        self.apply(instr.INSTR_CNOT, [q1, q2])
        self.apply(instr.INSTR_ROT_Y, q1, angle=-np.pi / 2)
        self.apply(instr.INSTR_ROT_X, q2, angle=np.pi / 2)
        yield self.run()


class BellMeasurementProgram(QuantumProgram):
    """Program for Bell state measurement on two qubits"""
    default_num_qubits = 4
    def program(self):
        q1, q2, q3, q4 = self.get_qubit_indices(4)
        self.apply(instr.INSTR_CNOT, [q3, q4])
        self.apply(instr.INSTR_H, q3)
        self.apply(instr.INSTR_MEASURE, q3, output_key="M3")
        self.apply(instr.INSTR_MEASURE, q4, output_key="M4")
        yield self.run()


class ClientProgram(QuantumProgram):
    """Simple program for single qubit measurement after Hadamard gate"""
    default_num_qubits = 1
    def program(self):
        q1 = self.get_qubit_indices(1)
        self.apply(instr.INSTR_H, q1)
        self.apply(instr.INSTR_MEASURE, q1, output_key="M5")
        yield self.run()


class ServerProgram(QuantumProgram):
    """Simple program to measure two qubits"""
    default_num_qubits = 2
    def program(self):
        q1, q2 = self.get_qubit_indices(2)
        self.apply(instr.INSTR_MEASURE, q1, output_key="M1")
        self.apply(instr.INSTR_MEASURE, q2, output_key="M2")
        yield self.run()


class M5XServerProgram(QuantumProgram):
    default_num_qubits = 2
    def program(self):
        q1, q2 = self.get_qubit_indices(2)
        self.apply(instr.INSTR_X, q2)
        self.apply(instr.INSTR_MEASURE, q1, output_key="M1")
        self.apply(instr.INSTR_MEASURE, q2, output_key="M2")
        yield self.run()


class Collect1Program(QuantumProgram):
    default_num_qubits = 3
    def program(self):
        q1, q2, q3 = self.get_qubit_indices(3)
        self.apply(instr.INSTR_CNOT, [q1, q3])
        self.apply(instr.INSTR_H, q1)
        yield self.run()


class Collect2Program(QuantumProgram):
    default_num_qubits = 3
    def program(self):
        q1, q2, q3 = self.get_qubit_indices(3)
        self.apply(instr.INSTR_CNOT, [q2, q3])
        self.apply(instr.INSTR_H, q2)
        yield self.run()

class XmeasureProgram(QuantumProgram):
    default_num_qubits = 2
    def program(self):
        q1, q2 = self.get_qubit_indices(2)
        self.apply(instr.INSTR_H, q1)
        self.apply(instr.INSTR_H, q2)
        yield self.run()

#
# 8) BellMeasurementProtocol, CorrectionProtocol classes
#
def memory_status(node):
    """Display the memory state of a node"""
    for pos in node.qmemory.mem_positions:
        print(f"Position {pos} is_empty: {pos.is_empty}")

class BellMeasurementProtocol(NodeProtocol):
    
    def __init__(self, node, shot_times_list, band_times_list, server_times_list, serverlist, max_runs, flag):
        super().__init__(node)
        self.shot_times_list = shot_times_list
        self.band_times_list = band_times_list
        self.server_times_list = server_times_list
        self.serverlist = serverlist
        self.max_runs = max_runs  # Maximum runs for the experiment
        self.run_count = 0        # Counter for current run
        self.flag = flag
    
    def run(self):
        global global_finished_bell, global_finished_correction
        qubit_initialised = False
        entanglement_ready = False
        c_meas_results = None
        qubit_init_program = InitStateProgram()
        measure_program = BellMeasurementProgram()
        server = ServerProgram()
        m5xserver = M5XServerProgram()
        collect1 = Collect1Program()
        collect2 = Collect2Program()
        xmesure = XmeasureProgram()
       
        init = True
        b1 = []
        b2 = []
        b3 = []
        b4 = []
        b5 = []
        astate = 0
        start_time = ns.sim_time()
        self.node.qmemory.execute_program(qubit_init_program)
        
        while True:
            if not init:
                yield self.await_timer(10)
                start_time = ns.sim_time()
                self.node.qmemory.execute_program(qubit_init_program)
                init = True
                qubit_index_4 = 3
                qubit_4, = self.node.qmemory.pop(qubit_index_4)
                qapi.discard(qubit_4)
            expr = yield (self.await_program(self.node.qmemory) |
                          self.await_port_input(self.node.ports["qin_Alice"]))
            if expr.first_term.value:
                qubit_initialised = True
                s1 = ns.sim_time() - start_time
            else:
                entanglement_ready = True
                b1.append(ns.sim_time())
            if qubit_initialised and entanglement_ready:
                qubit_initialised = False
                entanglement_ready = False
                e1 = ns.sim_time()
                yield self.node.qmemory.execute_program(measure_program)
                s2 = ns.sim_time() - e1
                m3, = measure_program.output["M3"]
                m4, = measure_program.output["M4"]
                # Pop and discard qubits
                qubit_index_3 = 2
                qubit_index_4 = 3
                qubit_4, = self.node.qmemory.pop(qubit_index_4)
                qapi.discard(qubit_4)
                self.node.ports["cout_bob"].tx_output((m3, m4))
                astate = 1
            while astate == 1:
                expr = yield (self.await_port_input(self.node.ports["cin_bob"]) |
                              self.await_port_input(self.node.ports["qin_Alice"]))
                if expr.first_term.value:
                    c_meas_results, = self.node.ports["cin_bob"].rx_input().items
                else:
                    entanglement_ready = True
                    b2.append(ns.sim_time())
                if c_meas_results is not None:
                    e2 = ns.sim_time()
                    self.node.qmemory.execute_program(collect1)
                    astate = 10
            while astate == 10:
                expr = yield (self.await_program(self.node.qmemory) |
                              self.await_port_input(self.node.ports["qin_Alice"])) 
                if expr.first_term.value:
                    qubit_initialised = True
                    s3 = ns.sim_time() - e2
                else:
                    entanglement_ready = True
                    b2.append(ns.sim_time())
                if qubit_initialised and entanglement_ready:
                    qubit_initialised = False
                    entanglement_ready = False
                    e3 = ns.sim_time()
                    yield self.node.qmemory.execute_program(measure_program)
                    s4 = ns.sim_time() - e3
                    m3, = measure_program.output["M3"]
                    m4, = measure_program.output["M4"]
                    qubit_index_3 = 2
                    qubit_index_4 = 3
                    qubit_4, = self.node.qmemory.pop(qubit_index_4)
                    qapi.discard(qubit_4)
                    self.node.ports["cout_bob"].tx_output((m3, m4))
                    astate = 2
            while astate == 2:
                expr = yield (self.await_port_input(self.node.ports["cin_bob"]) |
                              self.await_port_input(self.node.ports["qin_Alice"]))
                if expr.first_term.value:
                    c_meas_results, = self.node.ports["cin_bob"].rx_input().items
                else:
                    entanglement_ready = True
                    b3.append(ns.sim_time())
                if c_meas_results is not None:
                    e4 = ns.sim_time()
                    self.node.qmemory.execute_program(collect1)
                    astate = 11
            while astate == 11:
                expr = yield (self.await_program(self.node.qmemory) |
                              self.await_port_input(self.node.ports["qin_Alice"]))
                if expr.first_term.value:
                    qubit_initialised = True
                    s5 = ns.sim_time() - e4
                else:
                    entanglement_ready = True
                    b3.append(ns.sim_time())
                if qubit_initialised and entanglement_ready:
                    qubit_initialised = False
                    entanglement_ready = False
                    e5 = ns.sim_time()
                    yield self.node.qmemory.execute_program(measure_program)
                    s6 = ns.sim_time() - e5
                    m3, = measure_program.output["M3"]
                    m4, = measure_program.output["M4"]
                    qubit_index_3 = 2
                    qubit_index_4 = 3
                    qubit_4, = self.node.qmemory.pop(qubit_index_4)
                    qapi.discard(qubit_4)
                    self.node.ports["cout_bob"].tx_output((m3, m4))
                    astate = 3
                    
            while astate == 3:
                expr = yield (self.await_port_input(self.node.ports["cin_bob"]) |
                              self.await_port_input(self.node.ports["qin_Alice"]))
                if expr.first_term.value:
                    c_meas_results, = self.node.ports["cin_bob"].rx_input().items
                else:
                    entanglement_ready = True
                    b4.append(ns.sim_time())

                if c_meas_results is not None:
                    e6 = ns.sim_time()
                    self.node.qmemory.execute_program(collect2)
                    astate = 12
            while astate == 12:
                expr = yield (self.await_program(self.node.qmemory) |
                              self.await_port_input(self.node.ports["qin_Alice"]))
                if expr.first_term.value:
                    qubit_initialised = True
                    s7 = ns.sim_time() - e6
                else:
                    entanglement_ready = True
                    b4.append(ns.sim_time())
                if qubit_initialised and entanglement_ready:
                    qubit_initialised = False
                    entanglement_ready = False
                    e7 = ns.sim_time()
                    yield self.node.qmemory.execute_program(measure_program)
                    s8 = ns.sim_time() - e7
                    m3, = measure_program.output["M3"]
                    m4, = measure_program.output["M4"]
                    qubit_index_3 = 2
                    qubit_index_4 = 3
                    qubit_4, = self.node.qmemory.pop(qubit_index_4)
                    qapi.discard(qubit_4)
                    self.node.ports["cout_bob"].tx_output((m3, m4))
                    astate = 4
            while astate == 4:
                expr = yield (self.await_port_input(self.node.ports["cin_bob"]) |
                              self.await_port_input(self.node.ports["qin_Alice"]))
                if expr.first_term.value:
                    c_meas_results, = self.node.ports["cin_bob"].rx_input().items
                else:
                    entanglement_ready = True
                    b5.append(ns.sim_time())

                if c_meas_results is not None:
                    e8 = ns.sim_time()
                    self.node.qmemory.execute_program(collect2)
                    e9 = ns.sim_time()
                    astate = 13
            while astate == 13:
                expr = yield (self.await_program(self.node.qmemory) |
                              self.await_port_input(self.node.ports["qin_Alice"]))
                if expr.first_term.value:
                    qubit_initialised = True
                    s9 = ns.sim_time() - e8
                else:
                    entanglement_ready = True
                    b5.append(ns.sim_time())
                if qubit_initialised and entanglement_ready:
                    qubit_initialised = False
                    entanglement_ready = False
                    e9 = ns.sim_time()
                    yield self.node.qmemory.execute_program(measure_program)
                    s10 = ns.sim_time() - e9
                    m3, = measure_program.output["M3"]
                    m4, = measure_program.output["M4"]
                    qubit_index_3 = 2
                    qubit_index_4 = 3
                    qubit_4, = self.node.qmemory.pop(qubit_index_3)  # Corrected index
                    qapi.discard(qubit_4)
                    self.node.ports["cout_bob"].tx_output((m3, m4))
                    astate = 5
            while astate == 5:
                expr = yield (self.await_port_input(self.node.ports["cin_bob"]))
                if expr.first_term.value:
                    c_meas_results, = self.node.ports["cin_bob"].rx_input().items
                if c_meas_results is not None:
                    e10 = ns.sim_time()
                    if self.flag == 1:
                        yield self.node.qmemory.execute_program(xmesure)
                    if c_meas_results == 1:
                        yield self.node.qmemory.execute_program(server)
                        m1, = server.output["M1"]
                        m2, = server.output["M2"]
                    else:
                        yield self.node.qmemory.execute_program(server)
                        m1, = server.output["M1"]
                        m2, = server.output["M2"]
                    s11 = ns.sim_time() - e10
                    self.serverlist.append((m1, m2))
                    end_time = ns.sim_time()
                    run_time = end_time - start_time
                    band_time = (b1[0]-start_time) + (b2[0]-b1[-1]) + (b3[0]-b2[-1]) + (b4[0]-b3[-1]) + (b5[0]-b4[-1])
                    self.band_times_list.append(band_time)
                    self.shot_times_list.append(run_time)
                    self.server_times_list.append(s1+s2+s3+s4+s5+s6+s7+s8+s9+s10+s11)
                    b1 = []
                    b2 = []
                    b3 = []
                    b4 = []
                    b5 = []
                    self.send_signal(Signals.SUCCESS, 0)
                    self.run_count += 1
                    if self.run_count < self.max_runs:
                        qubit_initialised = False
                        entanglement_ready = False
                        init = False
                        c_meas_results = None
                        astate = 0
                        qubit_index_1 = 0
                        qubit_index_2 = 1
                        qubit_index_3 = 2
                        qubit_index_4 = 3
                    else:
                        global_finished_bell = True
                        if global_finished_bell and global_finished_correction:
                            ns.sim_stop()


class CorrectionProtocol(NodeProtocol):
    """Protocol for Bob's side. Receives Alice's measurement results, executes necessary feed-forward operations, 
    and then performs measurements."""
    def __init__(self, node, clientlist, telelist, max_runs, parameter):
        super().__init__(node)
        self.clientlist = clientlist
        self.parameter = parameter
        self.telelist = telelist
        self.max_runs = max_runs  # Maximum number of runs
        self.run_count = 0        # Current run counter
        
    def run(self):
        global global_finished_bell, global_finished_correction
        port_alice = self.node.ports["cin_alice"]
        port_bob = self.node.ports["qin_Bob"]
 
        entanglement_ready = False
        meas_results = None
        m5_result = None
        program = ClientProgram()
        state = 0
        angle = self.parameter
        
        while True:
            expr = yield (self.await_port_input(port_alice) | self.await_port_input(port_bob))
            if expr.first_term.value:
                meas_results, = port_alice.rx_input().items
            else:
                entanglement_ready = True

            if meas_results is not None and entanglement_ready:
                angle = self.parameter
                if meas_results[0] == 1:
                    angle = angle + np.pi
                if meas_results[1] == 1:
                    angle = -angle
                self.node.qmemory.execute_instruction(instr.INSTR_ROT_Z, [0], angle=angle)
                yield self.await_program(self.node.qmemory)
                yield self.node.qmemory.execute_program(program)
                qubit_index_5 = 0
                qubit_5, = self.node.qmemory.pop(qubit_index_5)  # Extract from memory
                qapi.discard(qubit_5)
                m5, = program.output["M5"]
                m5_result = m5
                tele1 = meas_results
                self.node.ports["cout_alice"].tx_output((m5))
                entanglement_ready = False
                meas_results = None
                state = 1

            while state == 1:
                expr = yield (self.await_port_input(port_alice) |
                              self.await_port_input(port_bob))
                if expr.first_term.value:
                    meas_results, = port_alice.rx_input().items
                else:
                    entanglement_ready = True

                if meas_results is not None and entanglement_ready:
                    angle = 0
                    if m5_result == 1:
                        angle = np.pi
                    if meas_results[0] == 1:
                        angle = angle + np.pi
                    if meas_results[1] == 1:
                        angle = -angle
                    self.node.qmemory.execute_instruction(instr.INSTR_ROT_Z, angle=angle)
                    yield self.await_program(self.node.qmemory)
                    yield self.node.qmemory.execute_program(program)
                    qubit_index_5 = 0
                    qubit_5, = self.node.qmemory.pop(qubit_index_5)  # Extract from memory
                    qapi.discard(qubit_5)
                    m5, = program.output["M5"]
                    m5_2 = m5
                    tele2 = meas_results
                    self.node.ports["cout_alice"].tx_output((m5))
                    entanglement_ready = False
                    meas_results = None
                    state = 2

            while state == 2:
                expr = yield (self.await_port_input(port_alice) |
                              self.await_port_input(port_bob))
                if expr.first_term.value:
                    meas_results, = port_alice.rx_input().items
                else:
                    entanglement_ready = True
                if meas_results is not None and entanglement_ready:
                    angle = 0
                    if m5_result == 1:
                        angle = np.pi
                    if meas_results[0] == 1:
                        angle = angle + np.pi
                    if meas_results[1] == 1:
                        angle = -angle
                    self.node.qmemory.execute_instruction(instr.INSTR_ROT_Z, angle=angle)
                    yield self.await_program(self.node.qmemory)
                    yield self.node.qmemory.execute_program(program)
                    qubit_index_5 = 0
                    qubit_5, = self.node.qmemory.pop(qubit_index_5)  # Extract from memory
                    qapi.discard(qubit_5)
                    m5, = program.output["M5"]
                    m5_3 = m5
                    tele3 = meas_results
                    self.node.ports["cout_alice"].tx_output((m5))
                    entanglement_ready = False
                    meas_results = None
                    state = 3

            while state == 3:
                expr = yield (self.await_port_input(port_alice) |
                              self.await_port_input(port_bob))
                if expr.first_term.value:
                    meas_results, = port_alice.rx_input().items
                else:
                    entanglement_ready = True
                if meas_results is not None and entanglement_ready:
                    angle = 0
                    if m5_result == 1:
                        angle = 0
                    if meas_results[0] == 1:
                        angle = angle + np.pi
                    if meas_results[1] == 1:
                        angle = -angle
                    self.node.qmemory.execute_instruction(instr.INSTR_ROT_Z, angle=angle)
                    yield self.await_program(self.node.qmemory)
                    yield self.node.qmemory.execute_program(program)
                    qubit_index_5 = 0
                    qubit_5, = self.node.qmemory.pop(qubit_index_5)  # Extract from memory
                    qapi.discard(qubit_5)
                    m5, = program.output["M5"]
                    m5_4 = m5
                    tele4 = meas_results
                    self.node.ports["cout_alice"].tx_output((m5))
                    entanglement_ready = False
                    meas_results = None
                    state = 4

            while state == 4:
                expr = yield (self.await_port_input(port_alice) |
                              self.await_port_input(port_bob))
                if expr.first_term.value:
                    meas_results, = port_alice.rx_input().items
                else:
                    entanglement_ready = True
                if meas_results is not None:
                    angle = 0
                    if m5_result == 1:
                        angle = np.pi
                    if meas_results[0] == 1:
                        angle = angle + np.pi
                    if meas_results[1] == 1:
                        angle = -angle
                    self.node.qmemory.execute_instruction(instr.INSTR_ROT_Z, angle=angle)
                    yield self.await_program(self.node.qmemory)
                    yield self.node.qmemory.execute_program(program)
                    m5, = program.output["M5"]
                    qubit_index_5 = 0
                    qubit_5, = self.node.qmemory.pop(qubit_index_5)  # Extract from memory
                    qapi.discard(qubit_5)
                    m5_5 = m5
                    tele5 = meas_results
                    self.clientlist.append((m5_result, m5_2, m5_3, m5_4, m5_5))
                    self.telelist.append((tele1, tele2, tele3, tele4, tele5))
                    self.node.ports["cout_alice"].tx_output((m5))
                    state = 5
                    self.send_signal(Signals.SUCCESS, 0)
                    self.run_count += 1
                    if self.run_count < self.max_runs:
                        qubit_initialised = False
                        entanglement_ready = False
                        init = False
                        c_meas_results = None
                        meas_results = None
                    else:
                        global_finished_correction = True
                        if global_finished_bell and global_finished_correction:
                            ns.sim_stop()

#
# 9) Network setup function modified to accept gate speed factor
#
def example_network_setup(dephase_rate, node_distance, T1, T2_ratio, client_T1, sge, dge, 
                          gate_speed_factor=1.0, client_gate_speed_factor=1, 
                          entanglement_fidelity=1.0, client_fidelity=1):
    """
    Set up the quantum network with specified parameters.
    
    Parameters:
    -----------
    dephase_rate : float
        Dephasing rate for quantum processors
    node_distance : float
        Distance between nodes in km
    T1 : float
        T1 relaxation time in ns
    T2_ratio : float
        Ratio of T2/T1 coherence times
    client_T1 : float
        Client T1 relaxation time in ns
    sge : float
        Single-qubit gate error rate
    dge : float
        Double-qubit gate error rate
    gate_speed_factor : float, optional
        Factor to scale gate execution speed (default 1.0)
    client_gate_speed_factor : float, optional
        Factor to scale client gate execution speed (default 1.0)
    entanglement_fidelity : float, optional
        Fidelity of entangled states (default 1.0)
    client_fidelity : float, optional
        Client quantum operation fidelity (default 1.0)
        
    Returns:
    --------
    Network
        Configured quantum network instance
    """
    alice = Node("Alice", qmemory=create_processor(dephase_rate=dephase_rate, T1=T1, 
                                                  T2_ratio=T2_ratio, sge=sge, dge=dge, 
                                                  gate_speed_factor=gate_speed_factor))
    bob = Node("Bob", qmemory=create_client_processor(dephase_rate=client_fidelity, T1=client_T1, 
                                                     T2_ratio=T2_ratio, sge=sge, dge=dge, 
                                                     gate_speed_factor=client_gate_speed_factor))
    network = Network("Teleportation_network")
    network.add_nodes([alice, bob])

    # Classical connections
    c_conn = ClassicalConnection(length=node_distance)
    network.add_connection(alice, bob, connection=c_conn, label="classical",
                           port_name_node1="cout_bob", port_name_node2="cin_alice")
    c_conn_reverse = ClassicalConnection(length=node_distance)
    network.add_connection(bob, alice, connection=c_conn_reverse, label="classical_reverse",
                           port_name_node1="cout_alice", port_name_node2="cin_bob")

    # Quantum channel connection
    q_conn = ExternalEntanglingConnection(length=node_distance, fidelity=entanglement_fidelity)
    port_ac, port_bc = network.add_connection(
        alice, bob,
        connection=q_conn,
        label="quantum",
        port_name_node1="qin_Alice",  # Unique port name for Alice
        port_name_node2="qin_Bob"     # Unique port name for Bob
    )

    # Forward Alice's "qin_Alice" → 'qin3' in qmemory
    alice.ports[port_ac].forward_input(alice.qmemory.ports['qin3'])
    # Forward Bob's "qin_Bob" → 'qin0' in qmemory
    bob.ports[port_bc].forward_input(bob.qmemory.ports['qin0'])

    return network

#
# 10) Simulation setup function modified to accept entanglement speed factor
#
def example_sim_setup(node_A, node_B, shot_times_list_alice, band_times_list_alice, server_times_list_alice,
                     max_runs, gate_speed_factor=1.0, entanglement_speed_factor=1.0,
                     serverlist=None, clientlist=None, telelist=None, parameter=0, flag=0):
    """
    Set up the simulation environment with the protocols and data collection.
    
    Parameters:
    -----------
    node_A, node_B : Node
        The network nodes to connect
    shot_times_list_alice, band_times_list_alice, server_times_list_alice : list
        Lists to store timing data
    max_runs : int
        Maximum number of protocol runs
    gate_speed_factor : float, optional
        Factor to scale gate execution speed (default 1.0)
    entanglement_speed_factor : float, optional
        Factor to scale entanglement generation speed (default 1.0)
    serverlist, clientlist, telelist : list, optional
        Lists to store measurement results
    parameter : float, optional
        Angle parameter for the protocol (default 0)
    flag : int, optional
        Flag to control protocol behavior (default 0)
        
    Returns:
    --------
    tuple
        (protocol_alice, protocol_bob, ext_source_protocol, dc)
    """
    def collect_fidelity_data(evexpr):
        protocol = evexpr.triggered_events[-1].source
        mem_pos = protocol.get_signal_result(Signals.SUCCESS)
        q1, = protocol.node.qmemory.pop(mem_pos)
        q2, = protocol.node.qmemory.pop(mem_pos+1)
        fidelity = qapi.fidelity(q2, ns.qubits.ketstates.s1, squared=True)
        qapi.discard(q2)
        return {"fidelity": fidelity}

    # Initialize protocols for Alice and Bob
    protocol_alice = BellMeasurementProtocol(node_A, shot_times_list_alice, band_times_list_alice, 
                                           server_times_list_alice, serverlist, max_runs, flag)
    protocol_bob = CorrectionProtocol(node_B, clientlist, telelist, max_runs, parameter=parameter)

    # Get the network that node_A belongs to
    network = node_A.supercomponent  # Teleportation_network

    # Get quantum connection with label="quantum"
    q_conn = network.get_connection(node_A, node_B, label="quantum")

    # Get QSource from subcomponents
    qsource = q_conn.subcomponents.get("AliceQSource")

    if qsource is None:
        raise ValueError("AliceQSource not found. Check subcomponent names.")

    # Protocol to gradually trigger QSource in EXTERNAL mode
    ext_source_protocol = ExternalSourceProtocol(
        node=node_A,
        qsource=qsource,  
        other_node=node_B,
        mem_pos_a=3,
        mem_pos_b=0,
        base_delay=1e9 / entanglement_speed_factor,  # Scale original base_delay
        extra_delay=1e6,        # Original extra_delay
        max_retries=int(1e3)
    )

    # Data collector
    dc = DataCollector(collect_fidelity_data)
    dc.collect_on(pydynaa.EventExpression(source=protocol_alice, event_type=Signals.SUCCESS.value))

    # Start protocols
    ext_source_protocol.start()
    protocol_alice.start()
    protocol_bob.start()

    return protocol_alice, protocol_bob, ext_source_protocol, dc

#
# 11) Run experiment function modified to handle gate and entanglement speed factors as variables
#
def convert_tuple_list_to_counts(tuple_list):
    """
    Convert a list of tuples to a count dictionary.
    
    Example: [(0,1), (1,0), (0,1), ...] -> {'01': count, '10': count, ...}
    
    Parameters:
    -----------
    tuple_list : list of tuple
        List of measurement outcome tuples
        
    Returns:
    --------
    dict
        Dictionary mapping outcome strings to counts
    """
    counts = {}
    for outcome_tuple in tuple_list:
        # Convert tuple to string "xy" (e.g., 0,1 -> "01")
        outcome_str = "".join(str(bit) for bit in outcome_tuple)
        # Increment the count
        counts[outcome_str] = counts.get(outcome_str, 0) + 1
    
    return counts

def evaluate_prob_difference(counts, ideal_state='10'):
    """
    Calculate the deviation of the measurement probability from the ideal value.
    
    Parameters:
    -----------
    counts : dict
        Dictionary of measurement outcome counts
    ideal_state : str, optional
        Expected ideal state (default '10')
        
    Returns:
    --------
    float
        Absolute difference between measured probability and ideal value (1.0)
    """
    total_shots = sum(counts.values())
    prob_ideal = counts.get(ideal_state, 0) / total_shots
    # Deviation from the true value (1.0)
    return abs(prob_ideal - 1.0)

import itertools

def run_experiment(num_runs, 
                   dephase_rates, client_fidelitys, distances, T1s, client_T1s, T2_ratios, 
                   sges, dges, gate_speed_factors, client_gate_speed_factors,
                   entanglement_fidelities, entanglement_speed_factors, max_runs, angle, flag):
    global global_finished_bell, global_finished_correction
    """
    Run simulations with varying parameters and collect results.

    Parameters
    ----------
    num_runs : int
        Number of simulations for each parameter combination.
    dephase_rates : list of float
        List of dephasing rates.
    client_fidelitys : list of float
        List of client fidelity values.
    distances : list of float
        List of distances (km).
    T1s : list of float
        List of T1 times (ns).
    client_T1s : list of float
        List of client T1 times (ns).
    T2_ratios : list of float
        List of T2/T1 ratios.
    sges : list of float
        List of single-qubit gate error rates.
    dges : list of float
        List of two-qubit gate error rates.
    gate_speed_factors : list of float
        List of gate speed factors.
    client_gate_speed_factors : list of float
        List of client gate speed factors.
    entanglement_fidelities : list of float
        List of entanglement fidelities.
    entanglement_speed_factors : list of float
        List of entanglement speed factors.
    max_runs : int
        Maximum number of protocol runs.
    angle : float
        Rotation angle parameter.
    flag : int
        Protocol behavior flag.

    Returns
    -------
    pandas.DataFrame
        DataFrame containing all collected data.
    """
    # List to store results
    results = []

    # Generate all combinations of parameters
    parameter_combinations = list(itertools.product(
        dephase_rates, distances, T1s, T2_ratios, client_T1s, client_fidelitys, client_gate_speed_factors, 
        sges, dges, gate_speed_factors, 
        entanglement_fidelities, entanglement_speed_factors
    ))

    for idx, (dephase_rate, distance, T1, T2_ratio, client_T1, client_fidelity, client_gate_speed_factor,
              sge, dge, gate_factor, 
              entanglement_fidelity, entanglement_factor) in enumerate(parameter_combinations, 1):

        # Calculate classical communication time based on distance
        cc_time = (1000 * distance / 200000 * 1e6)  # Assuming 200,000 km/s for light in fiber

        # Reset simulation
        ns.sim_reset()
        network = example_network_setup(dephase_rate, distance, T1, T2_ratio, client_T1, sge, dge, 
                                       gate_speed_factor=gate_factor, 
                                       client_gate_speed_factor=client_gate_speed_factor,
                                       entanglement_fidelity=entanglement_fidelity,
                                       client_fidelity=client_fidelity)

        node_a = network.get_node("Alice")
        node_b = network.get_node("Bob")
        shot_times_list_alice = []
        band_times_list_alice = []
        server_times_list_alice = []
        serverlist = []
        clientlist = []
        telelist = []

        protocol_alice, protocol_bob, ext_source_protocol, dc = \
            example_sim_setup(node_a, node_b,
                              shot_times_list_alice, band_times_list_alice, 
                              server_times_list_alice,
                              gate_speed_factor=gate_factor, 
                              entanglement_speed_factor=entanglement_factor,
                              serverlist=serverlist,
                              clientlist=clientlist,
                              telelist=telelist,
                              max_runs=max_runs,
                              parameter=angle,
                              flag=flag)

        # Run simulation
        ns.sim_run(1e15)
        global_finished_bell = False
        global_finished_correction = False

        # Process measurement results
        if flag == 0:
            for i in range(len(clientlist)):
                c = clientlist[i]
                s0, s1 = serverlist[i]
                
                # When client's 3rd bit (index 2) is 1, flip server's 1st bit (s0)
                if c[2] == 1:
                    s0 = 1 - s0  # Toggle 0 → 1, 1 → 0

                # When client's 5th bit (index 4) is 1, flip server's 2nd bit (s1)
                if c[4] == 1:
                    s1 = 1 - s1

                # Update serverlist with flipped results
                serverlist[i] = (s0, s1)
                
        if flag == 1:
            for i in range(len(clientlist)):
                c = clientlist[i]
                s0, s1 = serverlist[i]
                t = telelist[i]
                
                # When client's 2nd bit (index 1) is 1, flip server's 1st bit (s0)
                if c[1] == 1:
                    s0 = 1 - s0

                # When client's 4th bit (index 3) is 1, flip server's 2nd bit (s1)
                if c[3] == 1:
                    s1 = 1 - s1

                # When client's 1st bit (index 0) is 1, flip both s0 and s1
                if c[0] == 1:
                    s0 = 1 - s0
                    s1 = 1 - s1

                serverlist[i] = (s0, s1)

        # Collect fidelity data
        df_fidelity = dc.dataframe
        df_fidelity['distance'] = distance
        df_fidelity['gate_speed_factor'] = gate_factor
        df_fidelity['entanglement_speed_factor'] = entanglement_factor

        # Collect timing data
        shot_times_df = pandas.DataFrame({
            "distance": [distance] * len(shot_times_list_alice),
            "gate_speed_factor": [gate_factor] * len(shot_times_list_alice),
            "entanglement_speed_factor": [entanglement_factor] * len(shot_times_list_alice),
            "execution_time": shot_times_list_alice
        })
        band_times_df = pandas.DataFrame({
            "distance": [distance] * len(band_times_list_alice),
            "gate_speed_factor": [gate_factor] * len(band_times_list_alice),
            "entanglement_speed_factor": [entanglement_factor] * len(band_times_list_alice),
            "band_time": band_times_list_alice
        })
        server_times_df = pandas.DataFrame({
            "distance": [distance] * len(server_times_list_alice),
            "gate_speed_factor": [gate_factor] * len(server_times_list_alice),
            "entanglement_speed_factor": [entanglement_factor] * len(server_times_list_alice),
            "server_time": server_times_list_alice
        })
        meas_df = pandas.DataFrame({
            "distance": [distance] * len(server_times_list_alice),
            "gate_speed_factor": [gate_factor] * len(server_times_list_alice),
            "entanglement_speed_factor": [entanglement_factor] * len(server_times_list_alice),
            "server_result": serverlist,
            "client_result": clientlist
        })

        # Create counts_result
        counts_result = convert_tuple_list_to_counts(serverlist)
        diff_prob = evaluate_prob_difference(counts_result, ideal_state='10')

        # Save results
        for i in range(len(serverlist)):
            m = serverlist[i]
            c = clientlist[i]
            t = telelist[i]
            result = {
                "dephase_rate": dephase_rate,
                "client_fidelity": client_fidelity,
                "distance": distance,
                "T1": T1,
                "T2_ratio": T2_ratio,
                "client_T1": client_T1,
                "sge": sge,
                "dge": dge,
                "gate_speed_factor": gate_factor,
                "client_gate_speed_factor": client_gate_speed_factor,
                "entanglement_fidelity": entanglement_fidelity,
                "entanglement_speed_factor": entanglement_factor,
                "shot_time": shot_times_list_alice[i],
                "band_time": band_times_list_alice[i],
                "server_time": server_times_list_alice[i],
                "cc_time": cc_time,
                "server_result": m,
                "client_result": c,
                "teleport_result": t,
                "diff_prob": diff_prob,
                "angle": angle
            }
            results.append(result)

    # Convert to DataFrame
    results_df = pandas.DataFrame(results)
    
    return results_df


def count_tuple_frequencies(data):
    """
    Count frequencies of tuple outcomes with bit order reversed.
    
    Parameters:
    -----------
    data : list of tuple
        List of measurement outcome tuples
        
    Returns:
    --------
    dict
        Dictionary of outcome frequencies
    """
    freq_dict = {}
    for pair in data:
        key = "".join(map(str, pair[::-1]))  # Reverse the order
        if key in freq_dict:
            freq_dict[key] += 1
        else:
            freq_dict[key] = 1
    return freq_dict

def calculate_Z_cost(ans, shots):
    """
    Calculate the Z cost component of the optimization.
    
    Parameters:
    -----------
    ans : dict
        Dictionary of measurement outcomes and their counts
    shots : int
        Total number of shots
        
    Returns:
    --------
    float
        Calculated Z cost
    """
    cost = 0
    # 1
    cost += shots * (-0.4804)
    # Z0
    cost += (-ans.get("10", 0) + ans.get("01", 0) - ans.get("11", 0) + ans.get("00", 0)) * (0.3435)
    # Z1
    cost += (ans.get("10", 0) - ans.get("01", 0) - ans.get("11", 0) + ans.get("00", 0)) * (-0.4347)
    # Z0Z1
    cost += (-ans.get("10", 0) - ans.get("01", 0) + ans.get("11", 0) + ans.get("00", 0)) * (0.5716)
    cost = cost / shots

    return cost

def calculate_X_cost(ans, shots):
    """
    Calculate the X cost component of the optimization.
    
    Parameters:
    -----------
    ans : dict
        Dictionary of measurement outcomes and their counts
    shots : int
        Total number of shots
        
    Returns:
    --------
    float
        Calculated X cost
    """
    cost = 0
    cost += (-ans.get("10", 0) - ans.get("01", 0) + ans.get("11", 0) + ans.get("00", 0)) * (0.0910)
    cost = cost / shots

    return cost


def ZZ_cost(num_runs, 
            dephase_rates, client_fidelitys, distances, T1s, client_T1s, T2_ratios, 
            sges, dges, gate_speed_factors, client_gate_speed_factors,
            entanglement_fidelities, entanglement_speed_factors, shots, angle, flag=0):
    """
    Calculate the ZZ cost component by running the experiment.
    
    Parameters:
    -----------
    Multiple parameters for experiment configuration
        
    Returns:
    --------
    tuple
        (cost_value, total_shot_time)
    """
    results_df = run_experiment(
        num_runs=num_runs,
        dephase_rates=dephase_rates,
        client_fidelitys=client_fidelitys,
        distances=distances,
        T1s=T1s,
        T2_ratios=T2_ratios,
        client_T1s=client_T1s,
        sges=sges,
        dges=dges,
        gate_speed_factors=gate_speed_factors,
        client_gate_speed_factors=client_gate_speed_factors,
        entanglement_fidelities=entanglement_fidelities,
        entanglement_speed_factors=entanglement_speed_factors,
        max_runs=shots,
        angle=angle,
        flag=flag)
    
    serverlist = results_df['server_result'].tolist()
    result = count_tuple_frequencies(serverlist)
    x = calculate_Z_cost(result, shots)
    total_shot_time = results_df['shot_time'].sum()
    return x, total_shot_time

def XX_cost(num_runs, 
            dephase_rates, client_fidelitys, distances, T1s, client_T1s, T2_ratios, 
            sges, dges, gate_speed_factors, client_gate_speed_factors,
            entanglement_fidelities, entanglement_speed_factors, shots, angle, flag=1):
    """
    Calculate the XX cost component by running the experiment.
    
    Parameters:
    -----------
    Multiple parameters for experiment configuration
        
    Returns:
    --------
    tuple
        (cost_value, total_shot_time)
    """
    results_df = run_experiment(
        num_runs=num_runs,
        dephase_rates=dephase_rates,
        client_fidelitys=client_fidelitys,
        distances=distances,
        T1s=T1s,
        T2_ratios=T2_ratios,
        client_T1s=client_T1s,
        sges=sges,
        dges=dges,
        gate_speed_factors=gate_speed_factors,
        client_gate_speed_factors=client_gate_speed_factors,
        entanglement_fidelities=entanglement_fidelities,
        entanglement_speed_factors=entanglement_speed_factors,
        max_runs=shots,
        angle=angle,
        flag=flag)
        
    serverlist = results_df['server_result'].tolist()
    result = count_tuple_frequencies(serverlist)
    x = calculate_X_cost(result, shots)
    total_shot_time = results_df['shot_time'].sum()
    return x, total_shot_time

import time
import pandas as pd
import numpy as np
from scipy.optimize import minimize_scalar


def run_vqe_optimization_experiment(
    fidelity_factors,
    num_runs,
    dephase_rates,
    client_fidelitys,
    distances,
    T1s,
    T2_ratios,
    client_T1s,
    sges,
    dges,
    gate_speed_factors,
    client_gate_speed_factors,
    entanglement_fidelities,
    entanglement_speed_factors,
    shots,
    flag,
    tol=0.0015,
    bounds=(-np.pi, np.pi)
):
    """
    Run VQE optimization experiments with varying parameters.
    
    Parameters:
    -----------
    fidelity_factors : list of float
        Factors to scale gate fidelities
    num_runs : int
        Number of simulation runs per parameter set
    Additional parameters for experiment configuration
    tol : float, optional
        Optimization tolerance (default 0.0015)
    bounds : tuple, optional
        Angle bounds for optimization (default (-π, π))
        
    Returns:
    --------
    pandas.DataFrame
        Results of all optimization runs
    """
    results = []
    
    # Generate all parameter combinations
    parameter_combinations = list(itertools.product(
        dephase_rates, distances, T1s, T2_ratios, client_T1s, client_fidelitys, client_gate_speed_factors, 
        fidelity_factors, gate_speed_factors, 
        entanglement_fidelities, entanglement_speed_factors
    ))
    total_combinations = len(parameter_combinations)
    
    for idx, (dephase_rate, distance, T1, T2_ratio, client_T1, client_fidelity, 
              client_gate_speed_factor, fidelity_factor, gate_factor, 
              entanglement_fidelity, entanglement_factor) in enumerate(parameter_combinations, 1):
        
        sge = sges[0] * fidelity_factor
        dge = dges[0] * fidelity_factor
        print(f"\nSimulation {idx}/{total_combinations}:")
        print(f"dephase_rate={dephase_rate}, distance={distance} km, T1={T1} ns, T2={T1*T2_ratio} ns, "
              f"sge={sge}, dge={dge}, gate_speed_factor={gate_factor}, "
              f"entanglement_fidelity={entanglement_fidelity}, entanglement_speed_factor={entanglement_factor}")

        total_time_records = []

        def cost_func(angle):
            cost_zz, total_time_zz = ZZ_cost(
                num_runs=num_runs,
                dephase_rates=[dephase_rate],
                client_fidelitys=[client_fidelity],
                distances=[distance],
                T1s=[T1],
                T2_ratios=[T2_ratio],
                client_T1s=[client_T1],
                sges=[sge],
                dges=[dge],
                gate_speed_factors=[gate_factor],
                entanglement_fidelities=[entanglement_fidelity],
                entanglement_speed_factors=[entanglement_factor],
                client_gate_speed_factors=[client_gate_speed_factor],
                shots=shots,
                angle=angle,
                flag=0
            )
            cost_xx, total_time_xx = XX_cost(
                num_runs=num_runs,
                dephase_rates=[dephase_rate],
                client_fidelitys=[client_fidelity],
                distances=[distance],
                T1s=[T1],
                T2_ratios=[T2_ratio],
                client_T1s=[client_T1],
                sges=[sge],
                dges=[dge],
                gate_speed_factors=[gate_factor],
                entanglement_fidelities=[entanglement_fidelity],
                entanglement_speed_factors=[entanglement_factor],
                client_gate_speed_factors=[client_gate_speed_factor],
                shots=shots,
                angle=angle,
                flag=1
            )
            total_time = total_time_zz + total_time_xx
            total_time_records.append({total_time})
            return cost_zz + 2 * cost_xx

        start_time = time.perf_counter()
        result = minimize_scalar(cost_func, method='bounded', bounds=bounds, tol=tol)
        end_time = time.perf_counter()
        optimization_time = end_time - start_time

        final_energy = result.fun + 1 / 1.4172975
        final_angle = result.x
        nfev = result.nfev
        nit = result.nit
        total_time_sum = sum(next(iter(time_set)) for time_set in total_time_records)

        df = pd.DataFrame({
            'final_energy': [final_energy],
            'final_angle': [final_angle],
            'nfev': [nfev],
            'nit': [nit],
            'total_time': [total_time_sum],
            'fidelity_factors': [fidelity_factor],
            'num_runs': [num_runs],
            'dephase_rate': [dephase_rate],
            'client_fidelity': [client_fidelity],
            'distance': [distance],
            'T1': [T1],
            'T2_ratio': [T2_ratio],
            'client_T1': [client_T1],
            'sge': [sge],
            'dge': [dge],
            'gate_speed_factor': [gate_factor],
            'client_gate_speed_factor': [client_gate_speed_factor],
            'entanglement_fidelity': [entanglement_fidelity],
            'entanglement_speed_factor': [entanglement_factor],
            'shots': [shots],
            'flag': [flag]
        })
        results.append(df)

    results_df = pd.concat(results, ignore_index=True)
    return results_df